<a href="https://colab.research.google.com/github/KULL-Centre/_2023_Tesei_IDRome/blob/main/IDR_SVR_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preliminary information:**

This Colab notebook enables to predict the scaling exponent, $\nu$, and the conformational entropy per residue, $S_\text{conf}/N$, of an intrinsically disordered protein (IDP) or protein region (IDR) based on the amino acid sequence.

Predictions are generated by a support vector regression (SVR) model [1], which was trained on simulations of all the IDPs and IDRs of the human proteome performed using the CALVADOS model [2].

Amino acid sequences can be provided via (i) a single fasta file containing one or several entries, (ii) multiple fasta files, or (iii) pasting each sequence in separate input text boxes.

<b><font color='#FA003F'>How to cite this notebook:</font></b> If you use the $\nu$ values generated by the SVR model, we ask you to cite Tesei, Trolle et al. [1].
1. G. Tesei, A. I. Trolle, N. Jonsson, J. Betz, F. E. Knudsen, F. Pesce, K. E. Johansson, K. Lindorff-Larsen __Conformational ensembles of the human intrinsically disordered proteome__ _Nature._ 2024 626:897–904 2023.05.08.539815 DOI: https://doi.org/10.1038/s41586-023-07004-5
2. G. Tesei and K. Lindorff-Larsen __Improved predictions of phase behaviour of intrinsically disordered proteins by tuning the interaction range [version 2; peer review: 2 approved]__ _Open Research Europe_ 2023 2(94) DOI: https://doi.org/10.12688/openreseurope.14967.2
---


In [3]:
#@title <b>Preliminary operations</b>
import subprocess
subprocess.run( 'pip install wget localcider==0.1.18'.split() )
subprocess.run('pip uninstall scikit-learn -y'.split())
subprocess.run('pip install -U scikit-learn'.split())
import numpy as np
import itertools
from localcider.sequenceParameters import SequenceParameters
import wget
import sys
import os
from joblib import dump, load
import pandas as pd
from google.colab import files
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output

def calc_seq_prop(seq,residues,Nc,Cc,Hc):
    seq = list(seq).copy()
    fasta_kappa = np.array(seq.copy())
    N = len(seq)
    r = residues.copy()

    # calculate properties that do not depend on charges
    fK = sum([seq.count(a) for a in ['K']])/N
    fR = sum([seq.count(a) for a in ['R']])/N
    fE = sum([seq.count(a) for a in ['E']])/N
    fD = sum([seq.count(a) for a in ['D']])/N
    faro = sum([seq.count(a) for a in ['W','Y','F']])/N
    mean_lambda = np.mean(r.loc[seq].lambdas)

    pairs = np.array(list(itertools.combinations(seq,2)))
    pairs_indices = np.array(list(itertools.combinations(range(N),2)))
    # calculate sequence separations
    ij_dist = np.diff(pairs_indices,axis=1).flatten().astype(float)
    # calculate lambda sums
    ll = r.lambdas.loc[pairs[:,0]].values+r.lambdas.loc[pairs[:,1]].values
    # calculate SHD
    beta = -1
    shd = np.sum(ll*np.power(np.abs(ij_dist),beta))/N

    # fix charges
    if Nc == 1:
        r.loc['X'] = r.loc[seq[0]]
        r.loc['X','q'] = r.loc[seq[0],'q'] + 1.
        seq[0] = 'X'
        if r.loc['X','q'] > 0:
            fasta_kappa[0] = 'K'
        else:
            fasta_kappa[0] = 'A'
    if Cc == 1:
        r.loc['Z'] = r.loc[seq[-1]]
        r.loc['Z','q'] = r.loc[seq[-1],'q'] - 1.
        seq[-1] = 'Z'
        if r.loc['Z','q'] < 0:
            fasta_kappa[-1] = 'D'
        else:
            fasta_kappa[-1] = 'A'
    if Hc < 0.5:
        r.loc['H', 'q'] = 0
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'A'
    elif Hc >= 0.5:
        r.loc['H', 'q'] = 1
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'K'

    # calculate properties that depend on charges
    pairs = np.array(list(itertools.combinations(seq,2)))
    # calculate charge products
    qq = r.q.loc[pairs[:,0]].values*r.q.loc[pairs[:,1]].values
    # calculate SCD
    scd = np.sum(qq*np.sqrt(ij_dist))/N
    SeqOb = SequenceParameters(''.join(fasta_kappa))
    kappa = SeqOb.get_kappa()
    fcr = r.q.loc[seq].abs().mean()
    ncpr = r.q.loc[seq].mean()

    return pd.Series(data=[fK,fR,fE,fD,faro,scd,shd,kappa,fcr,mean_lambda,ncpr],
                 index=['fK','fR','fE','fD','faro','SCD','SHD','kappa','FCR','mean_lambda','NCPR'])

aa = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

url = 'https://github.com/KULL-Centre/_2023_Tesei_IDRome/blob/main'

if os.path.exists('svr_model_nu.joblib') == False:
    wget.download(url+'/svr_models/svr_model_nu.joblib?raw=true')
if os.path.exists('svr_model_SPR.joblib') == False:
    wget.download(url+'/svr_models/svr_model_SPR.joblib?raw=true')
if os.path.exists('residues.csv') == False:
    wget.download(url+'/md_simulations/data/residues.csv?raw=true')

model_nu = load('svr_model_nu.joblib')
model_spr = load('svr_model_SPR.joblib')
features_nu = ['SCD','SHD','kappa','FCR','mean_lambda']
features_spr = ['SCD','SHD','mean_lambda']

residues = pd.read_csv('residues.csv')
residues = residues.set_index('one')

fasta_dict = {}
df = pd.DataFrame(columns=['nuSVR','SconfSVR/N (kB)','mean_lambda','SHD','SCD','kappa','FCR','NCPR',
                           'fK','fR','fE','fD','faro'])

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [1]:
import pandas as pd

# Read your sequences from text file
with open('all_sequence_SOPA_9aa.txt', 'r') as f:
    sequences = [line.strip() for line in f if line.strip()]

# Create a dataframe with just the sequences
df = pd.DataFrame({
    'seq_name': [f'Seq_{i+1}' for i in range(len(sequences))],
    'fasta': sequences
})

# Save as CSV
df.to_csv('all_sequence_SOPA_9aa.csv', index=False)

In [8]:
# Initialize a dictionary to store FASTA sequences
fasta_dict = {}
current_upload = []

# Open and read the CSV file
with open('all_sequence_SOPA_9aa.csv', 'r') as f:
    # First, check if this is a CSV with headers by looking at the first line
    first_line = f.readline().strip()

    # Reset file pointer to beginning
    f.seek(0)

    # If it's a CSV with headers (like IDRome_DB.csv)
    if ',' in first_line:
        import pandas as pd
        df = pd.read_csv('all_sequence_SOPA_9aa.csv')

        # Check if 'fasta' column exists
        if 'fasta' in df.columns:
            for index, row in df.iterrows():
                seq_name = row.get('seq_name', f'Seq_{index}')
                fasta_dict[seq_name] = row['fasta']
                current_upload.append(seq_name)
        else:
            # If no 'fasta' column, assume the first column contains sequences
            column_name = df.columns[0]
            for index, row in df.iterrows():
                seq_name = f'Seq_{index}'
                fasta_dict[seq_name] = row[column_name]
                current_upload.append(seq_name)
    else:
        # If the file is a simple list of sequences, one per line
        for i, line in enumerate(f):
            line = line.strip()
            if line:  # Skip empty lines
                seq_name = f'Seq_{i+1}'
                fasta_dict[seq_name] = line
                current_upload.append(seq_name)

# Check sequences
for name in current_upload:
    sequence = fasta_dict[name]
    valid_aa = set("ACDEFGHIKLMNPQRSTVWY")
    if not all(aa in valid_aa for aa in sequence):
        print(f"Warning: Sequence {name} contains invalid amino acid characters!")
    else:
        print(f"Sequence {name} is valid with length {len(sequence)}")

print(f"Loaded {len(current_upload)} sequences")

Sequence Seq_1 is valid with length 9
Sequence Seq_2 is valid with length 9
Sequence Seq_3 is valid with length 9
Sequence Seq_4 is valid with length 9
Sequence Seq_5 is valid with length 9
Sequence Seq_6 is valid with length 9
Sequence Seq_7 is valid with length 9
Sequence Seq_8 is valid with length 9
Sequence Seq_9 is valid with length 9
Sequence Seq_10 is valid with length 9
Sequence Seq_11 is valid with length 9
Sequence Seq_12 is valid with length 9
Sequence Seq_13 is valid with length 9
Sequence Seq_14 is valid with length 9
Sequence Seq_15 is valid with length 9
Sequence Seq_16 is valid with length 9
Sequence Seq_17 is valid with length 9
Sequence Seq_18 is valid with length 9
Sequence Seq_19 is valid with length 9
Sequence Seq_20 is valid with length 9
Sequence Seq_21 is valid with length 9
Sequence Seq_22 is valid with length 9
Sequence Seq_23 is valid with length 9
Sequence Seq_24 is valid with length 9
Sequence Seq_25 is valid with length 9
Sequence Seq_26 is valid with leng

In [ ]:
#@title <b>Input sequence(s)</b>
#@markdown Or paste a sequence and provide a name. This cell can be executed multiple times to register more sequences.
NAME = "" #@param {type:"string"}
SEQUENCE = "" #@param {type:"string"}

if NAME != "" and SEQUENCE != "":
    if " " in SEQUENCE:
        SEQUENCE = ''.join(SEQUENCE.split())
    fasta_dict[NAME] = SEQUENCE

    #check sequence
    for a in fasta_dict[NAME]:
        if a not in aa:
            print('WARNING: {} sequence contains a character not recognized as an aminoacid. This sequence will be ignored'.format(name))
            del fasta_dict[NAME]

else:
    print('No NAME and/or SEQUENCE provided. Upload fasta files with the cell above or paste a sequence at the time here.')

In [9]:
#@title <b>Define charge states</b>
#@markdown Define charge states:
charged_N_terminal_amine = False #@param {type:"boolean"}
charged_C_terminal_carboxyl = True #@param {type:"boolean"}
charged_histidine = True #@param {type:"boolean"}
Nc = 1 if charged_N_terminal_amine == True else 0
Cc = 1 if charged_C_terminal_carboxyl == True else 0

if charged_histidine == True:
    print('Define pH and pKa to set the charge of Histidines according to the Henderson-Hasselbalch equation.')
    pH = input('Enter pH value: ')
    pH = float(pH)
    pKa = input('Enter pKa value: ')
    pKa = float(pKa)
    Hc = 1/(1+10**(pH-pKa))
if charged_histidine == False:
    Hc = 0

Define pH and pKa to set the charge of Histidines according to the Henderson-Hasselbalch equation.
Enter pH value: 7
Enter pKa value: 7


In [10]:
#@title <b>Predict $\nu$ and $S_\text{conf}/N$
#@markdown Use this cell to calculate sequence features and predict the scaling exponent, $\nu$, and the conformational entropy per residue, $S_\text{conf}/N$. Results will be downloaded in a csv file.

f = IntProgress(min=0, max=len(fasta_dict), description='Progress:', bar_style='warning')
display(f)

for k in fasta_dict.keys():
    res = calc_seq_prop(fasta_dict[k],residues,Nc,Cc,Hc)
    nu = np.around(model_nu.predict(res.loc[features_nu].values.reshape(1, -1))[0],3)
    spr = np.around(model_spr.predict(res.loc[features_spr].values.reshape(1, -1))[0],3)
    df.loc[k,'nuSVR'] = nu
    df.loc[k,'SconfSVR/N (kB)'] = spr
    df.loc[k,res.index.values] = np.around(res.loc[res.index.values].values,3)
    f.value += 1

clear_output()
df.to_csv('svr_pred.csv',index_label='name')
files.download('svr_pred.csv')

df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,seq_name,fasta,nuSVR,SconfSVR/N (kB),fK,fR,fE,fD,faro,SCD,SHD,kappa,FCR,mean_lambda,NCPR
0,Seq_1,VTLAERLAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Seq_2,GAAAAAAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Seq_3,EELVEEVLG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Seq_4,DAEAAADAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Seq_5,VVTDASKVK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Seq_496,NaN,NaN,0.615,10.691,0.0,0.0,0.000,0.000,0.111,0.000,1.422,1.000,0.111,0.381,-0.111
Seq_497,NaN,NaN,0.608,10.750,0.0,0.0,0.222,0.000,0.000,0.461,0.875,0.531,0.333,0.244,-0.333
Seq_498,NaN,NaN,0.649,10.847,0.0,0.0,0.222,0.333,0.111,2.899,0.939,0.101,0.667,0.242,-0.667
Seq_499,NaN,NaN,0.612,10.670,0.0,0.0,0.000,0.111,0.000,0.314,1.802,0.868,0.222,0.484,-0.222


In [11]:
import pandas as pd
import numpy as np

def normalize(series):
    """将数据归一化到0-1范围"""
    return (series - series.min()) / (series.max() - series.min())

def phase_separation_score(df):
    """计算相分离综合得分"""
    # 创建归一化列
    df['norm_nu'] = normalize(1 - df['nuSVR'])
    df['norm_kappa'] = normalize(df['kappa'])
    df['norm_SCD'] = normalize(df['SCD'])
    df['norm_faro'] = normalize(df['faro'])
    df['norm_Sconf'] = normalize(1 - df['SconfSVR/N'])

    # 计算加权得分
    df['phase_sep_score'] = (
        0.35 * df['norm_nu'] +
        0.25 * df['norm_kappa'] +
        0.20 * df['norm_SCD'] +
        0.15 * df['norm_faro'] +
        0.05 * df['norm_Sconf']
    )

    return df

# 使用示例
df = pd.read_csv('svr_pred.csv')
df_with_scores = phase_separation_score(df)
# 高分表示更高的相分离倾向

KeyError: 'SconfSVR/N'